In [ ]:
#IMPORT
import random
import hashlib
from itertools import chain
import hashlib
import binascii
import sys

In [ ]:
#FUNZIONI DI UTILITA
def generate_seed(seed_init=0):
    if seed_init==0:
        set_random_seed()
        seed = initial_seed()
        return seed
    else:
        set_random_seed(seed_init)
        return seed_init

def expand_seed(seed_init):
    seed_str = str(seed_init)
    seeds = hashlib.sha256(seed_str.encode()).digest()
    
    seed_1 = seeds[:16] #i primi 16 byte
    seed_2 = seeds[16:] #gli ultimi 16 byte
    
    seed_1 = int.from_bytes(seed_1, 'big')
    
    seed_2 = int.from_bytes(seed_2, 'big')
    
    return seed_1, seed_2
#t=8
# #w = 5

#0,2,4,5,7

#Creo Generatore g del Campo E
def get_generator(p, z, Fp):
    assert is_prime(p),"p is not prime!"
    assert (p-1)%z == 0, "z does not divide p-1!"
    alpha = Fp.primitive_element()
    #non ricordo perchè exp e g sono scelti secondo questa logica
    exp = (p-1)/z
    g = alpha**exp
    return g

def create_random_vector(domain, n_elems, seed):
    generate_seed(seed)
    return random_vector(domain, n_elems)

def create_random_matrix(domain, n_rows, n_columns, seed):
    generate_seed(seed)
    M = random_matrix(domain, n_rows, n_columns)
    return M

#Creo vettore errore con elementi in E e lunghezza V 
def create_vEn(g, n, Fz, Fp, seed):
    eta = create_random_vector(Fz, n, seed)
    e = vector(Fp, n)
    for i in range(0, n):#range check
        e[i] = (g**eta[i])
    return e

def componentwise_multiply(v1, v2):
    if len(v1) != len(v2):
        raise ValueError("I vettori devono avere la stessa dimensione")
    
    result = vector(GF(p), len(v1))  # Crea un nuovo vettore per i risultati
    for i in range(len(v1)):
        result[i] = v1[i] * v2[i]  # Moltiplica gli elementi corrispondenti
    
    return result

In [ ]:
#COSTRUZIONE ALBERO

def compute_node_childs(seed_parent, salt, i):
    nodes = str(seed_parent) + str(salt) + str(i)
    seed_sx, seed_dx = expand_seed(int(nodes))  
    return seed_sx, seed_dx

def SeedTreeLeaves(seed_parent, salt, i, lev, lev_leaf):
    seed_sx, seed_dx = compute_node_childs(seed_parent, salt, i)
    if lev == lev_leaf:
        return [seed_sx, seed_dx]
    else:
        return SeedTreeLeaves(seed_sx, salt, i*2, lev+1, lev_leaf) + SeedTreeLeaves(seed_dx, salt, i*2+1, lev+1, lev_leaf)

In [ ]:
#MIO MERKLE TREE

#[child_node]+ list
def find_index_proof(child_node, list):
    list = list+[child_node]
    if (child_node <= 2 and child_node % 2 == 0) or (child_node <=3):
        return list
    else:
        if(child_node%2==0):    
            parent_node = child_node/2
        else:
            parent_node = (child_node-1)/2
        if(parent_node%2==0):
            parent_node = parent_node+1
        else:
            parent_node = parent_node-1
        return find_index_proof(parent_node, list)  

def make_tree(nodes, level):
    if(level >= 0):
        new_nodes = create_new_level(nodes, level)
        nodes = new_nodes + nodes
        return make_tree(nodes, level-1)
    else:
        return nodes
        
def get_root(tree):
    return tree[0]

def get_proof(tree, node, list_index):
    if (node%2==0):
        node = node+1
    else:
        node = node-1
    list_index = find_index_proof(node, [])
    proof = []
    for i in range(len(list_index)):
        index = list_index[i]
        proof.append(tree[index])
    return proof

def verify_proof(root, list_proof, node):
    if len(list_proof)==0 :
        return node == root
    else :
        current_node = proof[0]
        temp = node+current_node ##è il giusto modo di concatenare?
        new_node = hashlib.sha256(temp.encode()).hexdigest()
        return verify_proof(root, list_proof[1:], new_node)
        

def create_new_level(nodes, level):
    new_nodes = []
    n_nodes = (2**level)-1
    for i in range(0, n_nodes, 2):
        temp = nodes[i]+nodes[i+1] ##è il giusto modo di concatenare?
        new_node = hashlib.sha256(temp.encode()).hexdigest()
        new_nodes.append(new_node)
    return new_nodes


In [ ]:
#CLASSE MERKLE TREE

class Node:
    def __init__(self, value=None):
        #self.index = index
        self.value = value
        self.sx = None
        self.dx = None
    
    def set_sx(self, sx):
        self.sx = sx

    def set_dx(self, dx):
        self.dx = dx
    
    def compute_node_value(self):
        to_hash = self.get_sx().get_value()+self.get_dx().get_value()
        self.value = hashlib.sha256(to_hash.encode()).hexdigest()
    
    def get_value(self):
        return self.value

    def get_sx(self):
        return self.sx

    def get_dx(self):
        return self.dx

class MerkleTree:
    def __init__(self, n_leaves):
        if(n_leaves%2==0):
            self.root = None
            self.n_leaves = n_leaves
            self.depth = log(n_leaves, 2)
            self.leaves = []
            self.tree = None
            self.ready = False
            self.full = False
        else:
            raise Exception('Il numero delle foglie deve essere una porenza di 2')

    #METODI PUBBLICI 
    def add_leaves(self, leaves):
        if isinstance(leaves, list) and len(leaves) == self.n_leaves:
            self.__add_leaves(leaves)
        else:
            raise Exception('Il numero delle foglie deve essere pari a {}'.format(self.n_leaves))
    
    def make_tree(self):
        if self.ready:
            self.__make_tree(self.depth, self.leaves)
            self.full = True
        else:
            raise Exception('Non ci sono foglie')

    def get_root(self):
        return self.root

    def get_proof(self, node):
        if self.full:
            if 1 <= node <= self.n_leaves:
                position = node + (2**self.depth) -1
                if position%2==0:
                    list_index = [position+1]
                else: 
                    list_index = [position-1]
                list_index = self.__find_index_proof(list_index)
                list_proof = self.__get_proof(list_index, [], self.tree)
                return list_proof
            else:
                raise Exception('Selezionare una foglia valida')
        else:
            raise Exception('Albero non creato')

    def verify_proof(self, node, list_proof):
        if self.full:  
            list_proof.append(node)
            return self.__verify_proof(list_proof)
        else:
            raise Exception('Albero non creato')  
    
    #METODI PRIVATI

    def __make_tree(self, depth, list_nodes):
        if(depth == 0):
            self.tree = list_nodes[0]
            self.root = self.tree.get_value()
        else:
            nodes = self.__create_new_level(list_nodes)
            self.__make_tree(depth-1, nodes)
            
    def __create_new_level(self, child_nodes):
        parents = []
        for i in range(0, len(child_nodes), 2):
            new_node = Node()
            new_node.set_sx(child_nodes[i])
            new_node.set_dx(child_nodes[i+1])
            new_node.compute_node_value()
            parents.append(new_node)
        return parents
    
    def __add_leaves(self, leaves):
        for i in range(len(leaves)):
            self.leaves.append(Node(leaves[i]))
        self.ready = True
    
    def __find_index_proof(self, list_nodes):
        child_node = list_nodes[0]
        if (child_node <= 2 and child_node % 2 == 0) or (child_node <=3):
            return list_nodes
        else:
            if(child_node %2==0):    
                parent_node = child_node /2
            else:
                parent_node = (child_node -1)/2
            if(parent_node%2==0):
                parent_node = parent_node+1
            else:
                parent_node = parent_node-1
            list_nodes.insert(0, parent_node)
            return self.__find_index_proof(list_nodes)          
    
    def __get_proof(self, list_index, list_proof, tree):
        if(len(list_index)==0):
            return list_proof
        else:
            if list_index[0]%2==0:
               list_index.pop(0)
               list_proof.append(tree.get_sx().get_value())
               return self.__get_proof(list_index, list_proof,  tree.get_dx())
            else:
                list_index.pop(0)
                list_proof.append(tree.get_dx().get_value())
                return self.__get_proof(list_index, list_proof, tree.get_sx())

    def __verify_proof(self, list_proof):
        while len(list_proof) > 1:
            ultimo = list_proof.pop()
            penultimo = list_proof.pop()
            to_hash= penultimo +ultimo
            hash=hashlib.sha256(to_hash.encode()).hexdigest()
            list_proof.append(hash)
        return list_proof[0] == self.get_root()
 

In [ ]:
#PARAMS
p = 31
Fp = GF(p)
n = 256
k = 52
r = n-k
t = 16
z = 3
Fz = GF(z)
lamb = 128
lev_leaf = log(t,2)
Fp_set = Set(Fp)
Fp_star = list(Fp_set.difference([0]))
print(Fp)
print(Fp_star)
msg = "buonasera"


In [ ]:
#GENERAZIONE CHIAVI
def key_gen():
    seed_sk = generate_seed() 
    seed_e , seed_pk = expand_seed(seed_sk)
    V = create_random_matrix(Fp, k, r, seed_pk) 
    g = get_generator(p, z, Fp)
    e = create_vEn(g, n, Fz, Fp, seed_e)
    s = e[0:r] + e[r:n]*V
    return seed_sk, seed_pk, s


In [ ]:
#FIRMA
def key_material_expansion(seed_sk):
    seed_e , seed_pk = expand_seed(seed_sk)
    V = create_random_matrix(Fp, k, r, seed_pk)
    eta = create_random_vector(Fz, n, seed_e)
    return V, eta

def compute_commitment(V, eta):
    seed_m = generate_seed() 
    seed_salt = generate_seed()
    salt = hashlib.sha256(str(seed_salt).encode()).digest()
    salt = int.from_bytes(salt, 'big')
    seeds = SeedTreeLeaves(seed_m, salt, 1, 0, lev_leaf)
    cmt0 = [] 
    cmt1 = []
    for i in range(0, t):
        seed_ui, seed_ei = expand_seed(seeds[i])
        eta_i = create_random_vector(Fz, n, seed_ei)
        sigma_i = eta-eta_i #sigma_i ∈ Fz^ns
        v = vector(Fp,n) #v∈ Fp^n
        g = get_generator(p, z, Fp)
        for j in range(0, n-1):
            v[j] = (g**sigma_i[j])  
        ui = create_random_vector(Fp, n, seed_ui)
        u = componentwise_multiply(v, ui)
        st = u[0:r] + u[r:n]*V
        commit0 = hashlib.sha256(( str(st) + str(sigma_i) + str(salt) + str(i) ).encode()).hexdigest()
        commit1 = hashlib.sha256(( str(seeds[i]) + str(salt) + str(i) ).encode()).hexdigest()
        cmt0.append(commit0) 
        cmt1.append(commit1)
    mt = MerkleTree(t)
    mt.add_leaves(cmt0)
    mt.make_tree()
    c0 = mt.get_root()
    input_c1 = ""
    for cmti in cmt1:
        input_c1 = input_c1+cmti
    c1 = hashlib.sha256(input_c1.encode()).hexdigest()
    return seeds, salt, c0, c1

def generate_first_challenge(c0, c1, msg, salt):
    #ritrasformo salt in stringa
    salt = salt.to_bytes((salt.bit_length() + 7) // 8, 'big').hex()
    input_ch1 = c0+c1+msg+salt
    seed_ch1 = hashlib.sha256(str(input_ch1).encode()).digest()
    seed_ch1 = int.from_bytes(seed_ch1, 'big')
    ch1 = create_random_vector(Fp_star, t, seed_ch1)
    return ch1

def generate_first_response(seeds, ch1):
    list_hash_yi=[]
    for seed_i, ch_i_1 in zip(seeds, ch1):
        seed_ui, seed_ei = expand_seed(seed_i)
        g = get_generator(p, z, Fp)
        ei = create_vEn(g, n, Fz, Fp, seed_ei)
        ui = create_random_vector(Fp, n, seed_ui)
        yi = seed_ui+(ch_i_1)*ei
        list_hash_yi.append(yi)
    print(list_hash_yi)




In [ ]:
seed_sk, seed_pk, s = key_gen()
V, eta = key_material_expansion(seed_sk)
seeds, salt, c0, c1 = compute_commitment(V, eta)
ch1 = generate_first_challenge(c0, c1, msg, salt)
generate_first_response(seeds, ch1)

#Beta appartiene a Fq*
(Beta1, ...., Betat) = GenCh2(c0, c1, Msg, Salt)
For i = 1, . . . , t:
    Compute y(i) = u′(i) + β(i)e′(i)
    Compute h(i) = Hash
Compute h = hash(h1,..,ht)

Generate (b1,.., bt)= GenCh2(c0,c1m beta1, .., Betat, h, msg, salt)

In [ ]:
seed_m = generate_seed() 
seed_salt = generate_seed()
salt = hashlib.sha256(str(seed_salt).encode()).digest()
print(salt.hex())
salt = int.from_bytes(salt, 'big')
print(salt)
salt = salt.to_bytes((salt.bit_length() + 7) // 8, 'big').hex()
print(salt)


In [ ]:
F = GF(p)
F_star = F.unit_group()
for a in F_star:
    print(a)

In [ ]:
p = 8
Fp = GF(p)
invertible = Fp.invertible_elements()
Fp_star = GF(len(invertible))(invertible)
seed = generate_seed()
v = create_random_vector(Fp_star, 10, seed)
print(v)